<a href="https://colab.research.google.com/github/Patrikwork/Course/blob/main/time_series_forecasting_with_machine_learning_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Forecasting using XGBoost
## Using Machine Learning to Forecast Energy Consumption


<img src="https://moodle.lut.fi/pluginfile.php/1/theme_maker_lab/logo/1697517856/LAB_eng_NEG.png" alt="drawing" width="350"/>



In this cell, we are setting up our environment by importing essential Python libraries that we will use throughout this notebook.

pandas: For data manipulation and analysis.
numpy: For numerical computing with support for large, multi-dimensional arrays and matrices.
matplotlib.pyplot: For creating static, interactive, and animated visualizations in Python.
seaborn: For making statistical graphics in Python. It is built on top of matplotlib and closely integrated with pandas data structures.
xgboost: An optimized distributed gradient boosting library designed to be highly efficient, flexible, and portable.
mean_squared_error from sklearn.metrics: To evaluate the accuracy of our model by calculating the mean squared error.
We also set a color palette for consistent plotting styles and apply the 'fivethirtyeight' style for all our plots for aesthetic purposes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
!curl -O https://raw.githubusercontent.com/Patrikwork/Course/main/PJME_hourly.csv
!head -n 5 /content/PJME_hourly.csv

## Types of Time Series Data

![](https://miro.medium.com/max/1400/1*V_RKPeIxCB9CS_2SsLyKXw.jpeg)

reference: https://engineering.99x.io/time-series-forecasting-in-machine-learning-3972f7a7a467

In this cell, we are performing the following actions:

We load the dataset into a pandas DataFrame. This dataset contains the energy consumption data we want to analyze and forecast.
We set the 'Datetime' column as the index of our DataFrame, which is essential for time series analysis since it allows us to manipulate the data based on time conveniently.
We ensure that the index is of type datetime by using the pd.to_datetime method. This step is crucial as it enables us to use time-based indexing and resampling techniques that are powerful tools for time series analysis.

In [ ]:
df = pd.read_csv('/content/PJME_hourly.csv')
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)

In [ ]:
df.plot(style='.',
        figsize=(15, 5),
        color=color_pal[0],
        title='PJME Energy Use in MW')
plt.show()

# Train / Test Split

In this cell, we are doing the following:

We split the dataset into two parts: one for training our model and another for testing its predictions. We are using '01-01-2015' as the cut-off date, with data before this date used for training and data on or after this date used for testing.
We then plot the training and test sets to visualize the split. The training set is the portion of the data that the model learns from, and the test set is the part that we use to evaluate the model's forecasting abilities.
We also draw a black dashed line at the '01-01-2015' cut-off date to clearly indicate the division between the training and test data on the plot.

In [ ]:
train = df.loc[df.index < '01-01-2015']
test = df.loc[df.index >= '01-01-2015']

fig, ax = plt.subplots(figsize=(15, 5))
train.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
test.plot(ax=ax, label='Test Set')
ax.axvline('01-01-2015', color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()

In this cell, we're focusing on a smaller slice of our data:

We filter the DataFrame to show only a single week of data, from '01-01-2010' to '01-08-2010'.
This filtered data is then plotted to give us a closer look at what a typical week looks like in our time series. This can help us understand the daily and weekly patterns and variations in the energy consumption data we're analyzing.

In [ ]:
df.loc[(df.index > '01-01-2010') & (df.index < '01-08-2010')] \
    .plot(figsize=(15, 5), title='Week Of Data')
plt.show()

# Feature Creation

In this cell, we define a function create_features which creates new columns in our DataFrame that represent various time components of the index. These are derived from the timestamp of each observation and include:

hour: The hour of the day.
dayofweek: The day of the week.
quarter: The quarter of the year.
month: The month of the year.
year: The year.
dayofyear: The day of the year.
dayofmonth: The day of the month.
weekofyear: The week of the year.
We then apply this function to our DataFrame, enriching it with these features which will be useful for our time series forecasting model to identify and learn patterns based on time.



In [ ]:
def create_features(df):
    """
    Create time series features based on time series index.
    """
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df

df = create_features(df)

# Visualize our Feature / Target Relationship

This cell produces a boxplot to visualize the distribution of energy consumption (PJME_MW) for each month. The following actions are performed:

We use seaborn's boxplot to create the plot, which shows the median, quartiles, and outliers for energy consumption in each month.
This visualization helps to identify seasonal trends and the variability in energy consumption throughout the year.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df, x='hour', y='PJME_MW')
ax.set_title('MW by Hour')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.boxplot(data=df, x='month', y='PJME_MW', palette='Blues')
ax.set_title('MW by Month')
plt.show()

# Create our Model

In this cell, we are preparing our dataset for the machine learning model:

We apply the create_features function to both the training and test sets to generate the time series features.
We define the features (FEATURES) that we will use to train our model and the target variable (TARGET) that we want to predict.
We then create X_train and y_train from the training set, which contain the features and target variable respectively.
Similarly, we create X_test and y_test from the test set. These will be used to evaluate the performance of our model after training.

In [ ]:
train = create_features(train)
test = create_features(test)

FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year']
TARGET = 'PJME_MW'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

In this cell, we are initializing and training our machine learning model:

We use the XGBRegressor from the xgboost library, which is a popular and powerful model for regression tasks that uses gradient boosting on decision trees.
We set various parameters for the model, such as n_estimators (the number of trees), max_depth, and learning_rate.
We also use early_stopping_rounds to prevent overfitting by stopping the training if the test set error doesn't improve for a given number of rounds.
The reg.fit method trains the model on our training data (X_train, y_train) and also evaluates it on both the training set and test set (X_test, y_test), providing verbose output every 100 rounds.


In [ ]:
reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',
                       n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:linear',
                       max_depth=3,
                       learning_rate=0.01)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

# Feature Importance

After training the model, this cell generates a plot to visualize the importance of each feature:

We create a DataFrame fi from the model's feature_importances_, which indicates how much each feature contributes to the model's predictions.
We then sort and plot these importances using a horizontal bar chart.
This visualization is crucial for understanding which features are most influential in the model's forecasts, and it can inform us about the relevance of different time components to the energy consumption pattern.

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

# Forecast on Test

Here, we use our trained model to make predictions and visualize them:

We add a new column prediction to the test set, which contains the model's predictions for the test period.
We merge these predictions back into the original DataFrame df so we can compare them with the actual values.
We plot the true energy consumption data and overlay the predictions to visually assess how well our model has learned the pattern.
A legend is added to help distinguish between the true data (Truth Data) and the model's predictions (Predictions).
The title 'Raw Data and Prediction' indicates that we are looking at the actual and forecasted values together.

In [ ]:
test['prediction'] = reg.predict(X_test)
df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
ax = df[['PJME_MW']].plot(figsize=(15, 5))
df['prediction'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Dat and Prediction')
plt.show()

This cell focuses on a detailed comparison of actual data and predictions for a specific week:

We filter the DataFrame df to visualize the actual (PJME_MW) and predicted (prediction) energy consumption values for the first week of April 2018.
We plot these values to see how closely the predictions match the actual data on a more granular level.
The legend differentiates between the 'Truth Data' (actual values) and 'Prediction' (model forecast), allowing us to visually assess the model's performance for this specific period.

In [ ]:
ax = df.loc[(df.index > '04-01-2018') & (df.index < '04-08-2018')]['PJME_MW'] \
    .plot(figsize=(15, 5), title='Week Of Data')
df.loc[(df.index > '04-01-2018') & (df.index < '04-08-2018')]['prediction'] \
    .plot(style='.')
plt.legend(['Truth Data','Prediction'])
plt.show()

# Score (RMSE & MAPE)

In the final cell, we calculate the model's performance metric:

We use the mean_squared_error function from sklearn.metrics, taking the square root of the result to get the Root Mean Squared Error (RMSE) score.
The RMSE score is a common measure of the accuracy of a regression model, indicating the standard deviation of the residuals (prediction errors).
We print out the RMSE score, formatted to two decimal places, which gives us a quantitative measure of how well our model is performing on the test set.

In [ ]:
score = np.sqrt(mean_squared_error(test['PJME_MW'], test['prediction']))
print(f'RMSE Score on Test set: {score:0.2f}')

To gain a clearer understanding of our model's prediction accuracy, we will compute the Mean Absolute Percentage Error (MAPE). This metric is more intuitive than RMSE because it expresses errors as a percentage of the actual values. Here's what we're doing:

We calculate the MAPE using the actual values from the test set and the predicted values from our model.
The MAPE provides the average absolute error in percentage terms, which is straightforward to interpret — a MAPE of 5% would mean that the average prediction error is 5% of the actual value.
We multiply the MAPE by 100 to convert it to a percentage, making it even easier to read and understand.
We aim for the MAPE to be as low as possible, with 0% representing perfect predictions.
This metric will help us quantify the performance of our model in terms that are more meaningful for business stakeholders and less technical users.

In [ ]:
mape_score = mean_absolute_percentage_error(y_test, test['prediction'])
print(f'MAPE Score on Test set: {mape_score * 100:.2f}%')


# Calculate Error
- Look at the worst and best predicted days

In [ ]:
test['error'] = np.abs(test[TARGET] - test['prediction'])
test['date'] = test.index.date
test.groupby(['date'])['error'].mean().sort_values(ascending=False).head(10)

In [ ]:
# Convert the Series to DataFrame
top_errors_by_date_df = top_errors_by_date.reset_index()

# Print the DataFrame
print(top_errors_by_date_df)

# Saving and Using the Trained Model Elsewhere (DON'T RUN THE CODE HERE)

After training your machine learning model, you may want to save it so that you can use it in a different environment or application. This involves two main steps: saving the model to a file and then loading it from that file elsewhere. Here's how to do it:

Save the Model: Use joblib to save the model to a file. This is efficient for models with large numpy arrays.


In [ ]:
import joblib

# Save the model to a file
joblib.dump(reg, 'model_filename.joblib')


Replace reg with the variable name of your trained model and 'model_filename.joblib' with the desired file name.

Transfer the Model File: Move the saved file to the new environment by your preferred method (e.g., USB, cloud storage, version control).

Load the Model: In the new environment, load the model from the file.

In [ ]:
# Load the model from the file
loaded_model = joblib.load('model_filename.joblib')

Use the Model: Make predictions using the loaded model.

In [ ]:
# Use the loaded model to make predictions
predictions = loaded_model.predict(data_to_predict)

Ensure that data_to_predict is preprocessed in the same way as the training data was.

Remember to replace data_to_predict with your actual data when making predictions.



# Next Steps
- More robust cross validation
- Add more features (weather forecast, holidays)